# **객체 탐지 (Object Detection)**
- **객체탐지**
    - Classification + Localization
    - 한 이미지에서 객체의 위치(Bounding Box, Localization)와 해당 객체의 종류(Classification)를 식별
    - 객체 탐지 알고리즘은 일반적으로 이미지를 입력으로 받고, 경계 상자와 객체 클래스 리스트를 출력
    - 경계 상자에 대해 그에 대응하는 예측 클래스와 클래스의 신뢰도(confidence)를 출력

- **Applications**
    - 자율 주행 자동차에서 다른 자동차와 보행자를 찾을 때
    - 의료 분야에서 방사선 사진을 사용해 종양이나 위험한 조직을 찾을 때
    - 제조업에서 조립 로봇이 제품을 조립하거나 수리할 때
    - 보안 산업에서 위협을 탐지하거나 사람 수를 셀 때

## **1. Metric**

### **1-1. Bounding Box**
- 이미지에서 하나의 객체 전체를 포함하는 가장 작은 직사각형

  <img src="https://miro.medium.com/max/850/1*KL6r494Eyfh3iYEXQA2tzg.png">

### **1-2. IoU(Intersection Over Union)**
- 실측값(Ground Truth)과 모델의 예측 값(Prediction)의 Bounding Box가 얼마나 겹치는지를 나타내는 지표

  <img src="https://pyimagesearch.com/wp-content/uploads/2016/09/iou_equation.png" width="300">

- IoU가 높을수록 모델의 Localization 성능이 높은 것을 의미

  <img src="https://pyimagesearch.com/wp-content/uploads/2016/09/iou_examples.png" width="400">

- 예시

  <img src="https://www.pyimagesearch.com/wp-content/uploads/2016/09/iou_stop_sign.jpg">

### **1-3. 정밀도(Precision) & 재현율(Recall)**
- 객체 탐지에서의 오탐(잘못 탐지한 경우)과 미탐(탐지하지 못한 경우)의 개념으로 사용

  <img src="https://www.researchgate.net/publication/366846732/figure/fig4/AS:11431281199334346@1697565652772/Confusion-matrix-for-the-meaning-of-TP-FN-FP-and-TN.png">

  - True Positive(TP): 실제로 True인데, 모델도 True라 예측한 경우
  - False Positive(FP): 실제로 True인데, 모델이 False라 판단한 경우
  - False Negative(FN): 실제로 False인데, 모델이 True라 예측한 경우
  - True Negative(TN): 실제로 False인데, 모델도 False라 판단한 경우

  $\qquad precision = \frac{TP}{TP \ + \ FP}$
  $\qquad recall = \frac{TP}{TP \ + \ FN}$

  - 모델이 잘못된 특징을 기반으로 객체를 예측할 경우 오분류할 가능성(FP)이 높아져 정밀도(precision)가 낮아짐 - 오탐
  - 모델이 엄격한 조건을 기반으로 객체를 예측할 경우 식별하지 못할 가능성(FN)이 높아져 재현율(recall)이 낮아짐 - 미탐

### **1-4. 정밀도-재현율 곡선(PR-Curve, Precision-Recall Curve) 및 AP(Average Precision)**
- PR-Curve는 기본적으로 Precision과 Recall간 관계 그래프에 해당
- Precision과 Recall은 Positive와 Negative를 구분 짓는 임계값(Threshold)와 직접적으로 연관되어 있으며, Threshold에 따라 서로 반비례한 관계를 가짐

  <img src="https://blog.kakaocdn.net/dn/mYTjE/btqYJ9SpI4u/4mXE8eerqEcTijMfCuFSOK/img.png">

  - Recall = 90/(90+30) = 0.75
  - Precision = 90/(90+10) = 0.9

  <img src="https://blog.kakaocdn.net/dn/ch3YCY/btqYDktFOuU/6KZM96cuQ6amfSt1JsH0X0/img.png">

  - Recall = 80/(80+20) = 0.8
  - Precision = 80/(80+20) = 0.8

  <img src="https://blog.kakaocdn.net/dn/ZsiJ2/btqYKaDKxvo/j8iKmdcL1aE0JNsYjYnZcK/img.png">

  - Recall = 70/(70+10) = 0.875
  - Precision = 70/(70+30) = 0.7

- PR-Curve는 이러한 Precision과 Recall간 관계에 따른 Threshold의 값들을 나타는 그래프에 해당

  <img src="https://images.datacamp.com/image/upload/v1674126967/Precision_recall_curve_5076395f1c.jpg">
- AP(Average Precision)의 경우 해당 PR-Curve의 아래 넓이를 의미하며 0~1 사이의 값에 해당하고 1에 가까울수록 모델의 성능이 우수
  - AP = 1의 경우 Recall과 Precision이 완전히 독립적이라는 것을 의미하며, 이는 FP, FN = 0 즉 오/미탐이 전혀 존재하지 않는 경우를 의미
  - 모델 정확도가 100%에 해당하는 것을 의미
- mAP(mean Average Precision)은 다수의 조건에서의 PR-Curve의 면적의 평균을 의미하며, IoU와 복합적으로 사용되며 object detection에서의 핵심 metric로 활용

### **1-5. mAP & IoU**
- AP가 특정 threshold에 따른 Precision과 Recall간 관계 그래프의 하단 면적에 해당하며, object detection에서의 threshold는 IoU가 활용
  - AP_0.5: IoU 0.5 이상을 정답으로 인정하는 경우
  - AP_0.95: IoU 0.95 이상을 정답으로 인정하는 경우
- 일반적으로 mAP로 표기된 경우 IoU 0.5부터 0.05씩 증가시키며 측정한 AP들의 평균을 의미하며 AP@[0.5:0.05:0.95]로 표기하기도 함

 ### **1-6. Localization 전략[NMS(Non-Maximum Suppression, 비최댓값 억제)]**
- 확률이 가장 높은 상자와 겹치는 상자들을 제거하여 가장 정확한 bounding box를 찾는 과정
  1. 확률 기준으로 모든 상자를 정렬하고 먼저 가장 확률이 높은 상자를 취함
  2. 각 상자에 대해 다른 모든 상자와의 IoU를 계산
  3. 특정 임곗값을 넘는 상자(많이 겹치는 상자)는 제거
- IoU 계산식 상 2개 bounding box의 교집합을 합집합으로 나누기 때문에 가장 정확한 box만을 제외하고 나머지 box 영역을 제거시켜 분모에 해당하는 합집합을 낮추기 위한 기법

  <img src="https://pyimagesearch.com/wp-content/uploads/2014/10/nms_fast_03.jpg">

## **2. 데이터셋(Dataset)**

### **2-1. Pascal VOC Dataset**
- 시기: 2005년부터 2012년까지 진행
- Major: VOC 2012 dataset
- Class: 20개
      background
      aeroplane
      bicycle
      bird
      boat
      bottle
      bus
      car
      cat
      chair
      cow
      diningtable
      dog
      horse
      motorbike
      person
      pottedplant
      sheep
      sofa
      train
      tvmonitor
- T/V/T 데이터 : 5,717 / 5,823 / 비공개
- 평균 이미지당 2.4개의 객체 존재

  <img src="https://paperswithcode.github.io/sotabench-eval/img/pascalvoc2012.png">

### **2-2. MS COCO Dataset**
- 시기: 2014년부터 최근까지 진행중
- Major: MS COCO 2017 dataset
- Class: 80개
      person
      bicycle
      car
      motorbike
      aeroplane
      bus
      train
      truck
      boat
      traffic light
      fire hydrant
      stop sign
      parking meter
      bench
      bird
      cat
      dog
      horse
      sheep
      cow
      elephant
      bear
      zebra
      giraffe
      backpack
      umbrella
      handbag
      tie
      suitcase
      frisbee
      skis
      snowboard
      sports ball
      kite
      baseball bat
      baseball glove
      skateboard
      surfboard
      tennis racket
      bottle
      wine glass
      cup
      fork
      knife
      spoon
      bowl
      banana
      apple
      sandwich
      orange
      broccoli
      carrot
      hot dog
      pizza
      donut
      cake
      chair
      sofa
      pottedplant
      bed
      diningtable
      toilet
      tvmonitor
      laptop
      mouse
      remote
      keyboard
      cell phone
      microwave
      oven
      toaster
      sink
      refrigerator
      book
      clock
      vase
      scissors
      teddy bear
      hair drier
      toothbrush
- T/V/T 데이터 : 118,000 / 5,000 / 41,000
- 평균 이미지당 7.2개의 객체 존재

<img src="https://cocodataset.org/images/coco-examples.jpg">



## **3. YOLO (You Only Look Once)**
- 대표적인 One-stage Object Detection 모델로 높은 속도 대비 높은 정확성을 가지는 실시간 객체 탐지 알고리즘에 해당
- 고정된 크기의 이미지(416x416 또는 608x608)를 입력 받아 여러 종류의 백본을 통과해 최종적으로 $w \times h \times M$ 행렬을 출력하는 형태로 구성
    - 이때, $M = B \times (C + 5)$에 해당하며, $B$ : 그리드 셀당 경계 상자 개수, $C$: 클래스 개수 에 해당
    - $(C + 5)$의 5 값의 경우 bounding box의 좌표정보($x$, $y$) 및 크기정보($w$, $h$)와 더불어 객체 포함 확률(Objectness Score, $p_0$)까지 bounding box와 class 외 총 5가지 값을 추가로 추론하기 위함

### **3-1. Object Detection 실습**
#### **(1) YOLOv3 모델 (TensorFlow)**

- 코드: https://github.com/zzh8829/yolov3-tf2

#### **(2) YOLOv5 모델 (PyTorch)**

* 코드: https://github.com/ultralytics/yolov5